# 교통카드 빅데이터 분석
- 정류장 이용량 동적 크롤링
- https://stcis.go.kr/wps/dashBoard.do
- https://blog.naver.com/ses7361/222268384948

## 1. 동적 크롤링(Ajax, JSON)
- https://daeguowl.tistory.com/89
- 자바스크립트로 동작되는 객체들은 지속적으로 데이터를 가져와 업데이트를 하기에,<br>
현재 데이터 기준이 아닌, 원본 데이터 주소를 찾아야 한다.<br>
먼저 해당 데이터를 불러오는 url을 찾는 것이 중요한데 network tab에서 찾을 수 있다.<br>
XHR을 선택하면 Ajax로만 활동하는 것을 확인할 수 있고, 거기서 해당 내용을 불러오는 url을 구성한다<br>
그리고 그 url을 통해 크롤링을 진행해야하며, 동적 데이터이기 때문에 json을 사용한다.

## 2. 기본 개념 정립
- <b><i>브라우저와 서버간 통신</b></i><br>
브라우저에서 웹페이지를 요청하거나 링크를 클릭하면 화면전환이 발생하는데 이것은 브라우저와 서버와의 통신에 의한 것이다.<br>
서버는 요청받은 페이지(HTML)을 반환하는데 이때 HTML 뿐만 아니라 CSS 및 javascript 파일들도 함께 반환한다.<br>
웹페이지가 반환되면 브라우저(웹클라이언트)는 page를 reload하여 화면을 보여준다<br>

- <b><i>Ajax(Asynchronous javascript and XML)</b></i><br>
Ajax는 자바스크립트를 이용해 비동기적으로 서버와 브라우저가 데이터를 교환하는 방식이다<br>
서버에서 한번 페이지를 return할 때는 HTML과 같은 정적인 파일 뿐 아니라 javascript와 같이 동적인 파일들도 함께 return한다.<br>
만약 페이지에서 일부만 지속적으로 갱신해야 한다면, 굳이 모든 파일을 다시 반환할 필요가 없다.<br>
그렇기 때문에 브라우저(웹클라이언트)는 Ajax request를 통해 동적인 부분을 요청하고 서버는 json을 통해 reload하는데 필요한 정보들을 반환해준다

  <img src="https://user-images.githubusercontent.com/46436843/56737669-de65f780-67a5-11e9-956a-18ac520730c6.png" width="600" ><br>
- <b><i>Ajax의 요청과 처리</b></i><br>
웹브라우저는 XMLHttpRequest 객체를 이용해 Ajax 요청을 보낸다.<br>
서버는 JSON 형태로 요청온 정보만을 담아 순수한 텍스트로 구성된 데이터 문자열을 반환한다.<br>
이 문자열을 객체화 하는 것을 역직렬화라고 하고 역직렬화를 위해 JSON.parse()를 이용한다.<br>

- <b><i>크롤러</b></i> : 인터넷에 있는 데이터를 자동 수집하는 프로그램<br>
- <b><i>정적 크롤링</b></i><br>
request: 웹사이트 접속 데이터를 받아오는 역할<br>
BeautifulSoup : 데이터를 HTML로 해석하는 역할<br>

- <b><i>동적 크롤링</b></i><br>
Ajax: 웹클라이언트에서 서버에 데이터를 요청하는 역할<br>
JSON: 웹서버로부터 반환되는 데이터<br>
JSON.parse() : 반환된 데이터를 사용할 수 있도록 객체화 해주는 역할

In [13]:
# url = "https://stcis.go.kr/org/indicator/cmmnZoneSggListAjax.do"
# Method type : POST
# parameter : 
import warnings
import requests
import json            #json import하기

warnings.filterwarnings(action='ignore')    # 경고메시지 무시

#custom_header을 통해 아닌 것 처럼 위장하기
custom_header = {
    'referer' : 'https://stcis.go.kr/wps/dashBoard.do',
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'  }

#해당 접속 사이트가 아닌 원본데이터가 오는 url 추적. network에서 가지고 온다.
url = "https://stcis.go.kr/org/indicator/cmmnZoneSggListAjax.do"

# parm = [{"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27720","zoneId":"27720","sggNm":"군위군","zoneNm":"군위군"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27200","zoneId":"27200","sggNm":"남구","zoneNm":"남구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27290","zoneId":"27290","sggNm":"달서구","zoneNm":"달서구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27710","zoneId":"27710","sggNm":"달성군","zoneNm":"달성군"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27140","zoneId":"27140","sggNm":"동구","zoneNm":"동구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27230","zoneId":"27230","sggNm":"북구","zoneNm":"북구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27170","zoneId":"27170","sggNm":"서구","zoneNm":"서구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27260","zoneId":"27260","sggNm":"수성구","zoneNm":"수성구"},
#         {"rtryCd":"27","rtryNm":"대구광역시","sggCd":"27110","zoneId":"27110","sggNm":"중구","zoneNm":"중구"}]

parm = {
    "upZoneSd":"27",
    "searchUseaDay":"2024-05-17"
}

req = requests.post(url, headers = custom_header, data=parm)    #custom_header를 사용하지 않으면 접근 불가

if req.status_code == requests.codes.ok:    
    print("접속 성공")
    data = json.loads(req.text)        #json에 반환된 데이터가 들어가 있다.
    # for rank in stock_data['data']:         #stock_data는 'data' key값에 모든 정보가 들어가 있다.
    #     print(rank['rank'], rank['symbolCode'], rank['name'], rank['tradePrice'])
    print(data)
else:
    print("Error code")

접속 성공
{'ajaxResult': 'succ', 'listZoneSgg': [{'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27720', 'zoneId': '27720', 'sggNm': '군위군', 'zoneNm': '군위군'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27200', 'zoneId': '27200', 'sggNm': '남구', 'zoneNm': '남구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27290', 'zoneId': '27290', 'sggNm': '달서구', 'zoneNm': '달서구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27710', 'zoneId': '27710', 'sggNm': '달성군', 'zoneNm': '달성군'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27140', 'zoneId': '27140', 'sggNm': '동구', 'zoneNm': '동구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27230', 'zoneId': '27230', 'sggNm': '북구', 'zoneNm': '북구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27170', 'zoneId': '27170', 'sggNm': '서구', 'zoneNm': '서구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27260', 'zoneId': '27260', 'sggNm': '수성구', 'zoneNm': '수성구'}, {'rtryCd': '27', 'rtryNm': '대구광역시', 'sggCd': '27110', 'zoneId': '27110', 'sggNm': '중구', 'zoneNm': '중구'}]}
